In [144]:
from itertools import product
from random import random, randint, choice, seed
from functools import reduce
import numpy as np
import math
from scipy import sparse
from copy import copy

In [145]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [146]:
NUM_POINTS, NUM_SETS, DENSITY = 100, 100, 0.3
x = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)

In [147]:
def fitness1(state,x):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [x.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def fitness2(state,x):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [x.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

In [148]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_POINTS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [149]:
first_state = [choice([True, False]) for _ in range(NUM_POINTS)]
print(first_state)
def print_state(state):
    return ['T' if item is True else 'F' if item is False else item for item in state]

[False, False, True, True, False, True, True, False, False, True, False, False, True, True, False, False, False, True, True, False, True, True, False, True, False, True, False, True, True, False, False, False, False, False, False, True, False, False, False, False, True, True, False, True, False, True, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, True, True, True, False, True, True, False, False, False, True, False, True, True, False, False, False, True, False, False, False, True, True, False, False, True, False, False, False, True, True, False, False, True, False, True]


# Tabu search

In [150]:
DIM_TABU_LIST=100
current_state = first_state
print("initial state:",print_state(current_state))
neighbours=[]
tabu_list=[DIM_TABU_LIST]
step_done=0
best_state_value=fitness2(current_state,x)
best_state = current_state

for _ in range(100):
    neighbours = [tweak(current_state) for _ in range(100)]
    max_neig = neighbours[0]
    for neig in neighbours:
        if neig not in tabu_list and fitness2(neig,x) >= fitness2(max_neig,x):
            step_done +=1
            max_neig=neig

    current_state=max_neig
    tabu_list.append(current_state)
    if fitness2(current_state,x) > best_state_value:
        best_state = current_state
        best_state_value = fitness2(current_state,x)

    if len(tabu_list) >= DIM_TABU_LIST:
        tabu_list.pop()

print("final state:  ", print_state(current_state))
print("all step done:", step_done)
print(fitness2(best_state,x))

initial state: ['F', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'T']
final state:   ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'

# Iterated local search

In [151]:
current_state = first_state
print("initial state:",print_state(current_state))
steady_state_cont = 0
steady_state_limit = 10
step_done=0
for _ in range(100):
    new_state = tweak(current_state)
    if fitness2(new_state,x) >= fitness2(current_state,x):
        step_done+=1
        steady_state_cont = 0
        current_state = new_state
    else:
        steady_state_cont+=1
        if steady_state_cont == steady_state_limit :
            temp_state = [choice([True, False]) for _ in range(NUM_POINTS)]
            step_done+=1
            if fitness2(temp_state,x) > fitness2(current_state,x):
                current_state=temp_state
            steady_state_cont=0
print("final state:  ", print_state(current_state))
print("all step done:", step_done)
print(fitness2(current_state,x))

initial state: ['F', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'T']
final state:   ['F', 'F', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F'

# Simulated annealing

In [152]:
current_state = first_state
print("initial state:",print_state(current_state))
cooling_rate=0.97
cost=fitness2(current_state,x)
best_solution=current_state
best_cost=cost
temperature=cooling_rate
last_valid=current_state
last_cost=fitness2(current_state,x)
step_done=1
while temperature > 0.001 :  
        new_state = tweak(current_state)
        score_new=fitness2(new_state,x)
        score_curr=fitness2(current_state,x)
        step_done+=2
        if  score_new>score_curr or random() < math.exp( -((2*score_curr[0]+score_curr[1])-(2*score_new[0]+score_new[1]))/ temperature):
            current_state = new_state
            current_energy =score_new
            if(score_new[0]==NUM_POINTS):
                    last_valid=current_state
                    last_cost=score_new
            if score_new>best_cost:
                best_solution = current_state
                best_cost = score_new
        
        temperature *= cooling_rate
if(best_cost<=last_cost):
    best_cost=last_cost
    best_solution=last_valid
print("final state:  ", print_state(current_state))
print("all step done:", step_done)
print(fitness2(current_state,x))

initial state: ['F', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'T']
final state:   ['F', 'F', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'F'